### Mean Variance optimization  of player selection with PyPortfolioOpt

* too many missing values; double-GW causing errors 
* better to account for this when brute-forcing combinations (maximizing sharpe instead of raw returns) than solving the optimization problem 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [4]:
pd.set_option('display.max_columns', None)

In [6]:
merged = pd.read_csv ('merged.csv')

C:\Users\Bryant Xia\AppData\Local\Temp\ipykernel_11688\2592697057.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  merged = pd.read_csv ('merged.csv')


In [7]:
merged.head ()

,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,2016-17,Aaron Cresswell,DEF,NaN,0,0,0,0,0.0,454,10,0,0,0.0,0.0,2016-08-15T19:00:00Z,0,4,0,0,0,0,1,0,14023,1.0,2.0,0.0,0,0,0,0,55,False,0,1
1,2016-17,Aaron Lennon,MID,NaN,0,0,6,0,0.3,142,3,0,0,0.9,8.2,2016-08-13T14:00:00Z,15,17,0,0,0,0,1,0,13918,1.0,1.0,0.0,1,0,0,0,60,True,0,1
2,2016-17,Aaron Ramsey,MID,NaN,0,0,5,0,4.9,16,8,3,0,3.0,2.2,2016-08-14T15:00:00Z,60,9,0,0,0,0,1,0,163170,4.0,3.0,23.0,2,0,0,0,80,True,0,1
3,2016-17,Abdoulaye Doucouré,MID,NaN,0,0,0,0,0.0,482,7,0,0,0.0,0.0,2016-08-13T14:00:00Z,0,13,0,0,0,0,1,0,1051,1.0,1.0,0.0,0,0,0,0,50,False,0,1
4,2016-17,Adam Forshaw,MID,NaN,0,0,3,0,1.3,286,6,1,0,0.3,2.0,2016-08-13T14:00:00Z,69,14,0,0,0,0,1,0,2723,1.0,1.0,0.0,1,0,0,0,45,True,1,1


In [25]:
id = []
for i in range (len(merged)):
    id.append (merged['season_x'][i]+"_"+str(merged['GW'][i]))
merged ['time_id'] = id

In [26]:
simplified = pd.DataFrame ({
    "time_id": merged ['time_id'],
    "ret": merged ['total_points'],
    "player_id": merged ['element']
})

simplified.head ()

,time_id,ret,player_id
0,2016-17_1,0,454
1,2016-17_1,1,142
2,2016-17_1,2,16
3,2016-17_1,0,482
4,2016-17_1,1,286


In [29]:
#control for double GW
count = 1
id_num = [1]
prev = simplified ['time_id'][0]
for i in range (1, len(simplified)):
    cur = simplified ['time_id'][i]
    if (cur!=prev):
        count+=1
    id_num.append (count)
    prev = cur
simplified ['time_id'] = id_num
simplified.head ()

,time_id,ret,player_id
0,1,0,454
1,1,1,142
2,1,2,16
3,1,0,482
4,1,1,286


In [30]:
simplified.tail ()

,time_id,ret,player_id
100744,231,0,441
100745,231,1,436
100746,231,0,538
100747,231,0,119
100748,231,1,70


In [32]:
alt_id = []
cur_idx = 1
players_ct = dict () # of player id duplicates for a single GW 
for i in range (len(simplified)):
    if (cur_idx != simplified['time_id'][i]):
        cur_idx += 1
        players_ct = dict ()
    player_id = simplified ['player_id'][i]
    if player_id in players_ct:
        players_ct [player_id] += 1
        alt_id.append (cur_idx+0.1*players_ct[player_id])
    else:
        players_ct [player_id] = 0
        alt_id.append (cur_idx)
simplified ['time_id'] = alt_id

In [33]:
simplified['time_id'].unique ()

array([  1. ,   2. ,   3. ,   4. ,   5. ,   6. ,   7. ,   8. ,   9. ,
        10. ,  11. ,  12. ,  13. ,  14. ,  15. ,  16. ,  17. ,  18. ,
        19. ,  20. ,  20.1,  21. ,  22. ,  23. ,  24. ,  25. ,  26. ,
        27. ,  28. ,  28.1,  29. ,  30. ,  30.1,  31. ,  31.1,  32. ,
        33. ,  34. ,  35. ,  36. ,  37. ,  38. ,  39. ,  40. ,  41. ,
        42. ,  43. ,  44. ,  45. ,  46. ,  47. ,  48. ,  49. ,  50. ,
        51. ,  52. ,  53. ,  53.1,  54. ,  55. ,  56. ,  57. ,  58. ,
        59. ,  60. ,  61. ,  62. ,  63. ,  64. ,  65. ,  66. ,  66.1,
        67. ,  68. ,  69. ,  69.1,  70. ,  71. ,  72. ,  73. ,  74. ,
        75. ,  76. ,  77. ,  78. ,  79. ,  80. ,  81. ,  82. ,  83. ,
        84. ,  85. ,  86. ,  87. ,  88. ,  89. ,  90. ,  91. ,  92. ,
        93. ,  94. ,  95. ,  96. ,  97. ,  98. ,  99. , 100. , 101. ,
       101.1, 102. , 103. , 104. , 105. , 106. , 107. , 108. , 108.1,
       109. , 110. , 110.1, 111. , 111.1, 112. , 113. , 114. , 115. ,
       116. , 117. ,

In [65]:
pivoted = simplified.pivot (index="time_id", columns="player_id", values="ret")

In [66]:
drops = []
for i in range (1, 738):
    if (np.mean (pivoted[i])<2):
        drops.append (i)

pivoted = pivoted.drop (columns = drops)

In [67]:
pivoted.head ()

player_id,6,11,12,14,16,18,22,23,38,40,42,45,54,62,67,69,70,76,77,78,80,83,84,86,91,93,97,100,106,109,111,113,115,118,122,123,126,127,128,132,134,138,141,142,143,148,150,154,155,159,162,164,166,168,172,184,187,191,198,199,200,202,203,209,210,211,212,213,215,217,219,221,224,225,230,231,232,233,234,235,239,240,245,247,251,252,254,255,256,257,259,262,265,266,268,270,271,274,276,277,278,280,282,284,296,298,300,301,302,305,306,313,315,326,336,338,340,342,343,345,351,353,357,359,362,364,366,367,368,370,376,388,390,394,396,403,409,411,417,419,423,425,431,432,435,437,449,450,465,468,475,493,494,558,570,588,681,696,697,699,700,701,702
time_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,0.0,6.0,NaN,0.0,2.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,0.0,2.0,5.0,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0,1.0,NaN,1.0,NaN,NaN,NaN,2.0,NaN,0.0,2.0,2.0,NaN,10.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,0.0,NaN,2.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2.0,NaN,NaN,7.0,0.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,9.0,NaN,0.0,2.0,NaN,NaN,NaN,2.0,7.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,0.0,NaN,NaN,2.0,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN,1.0,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,NaN,8.0,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,2.0,5.0,NaN,5.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,1.0,1.0,8.0,6.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,0.0,NaN,5.0,NaN,NaN,NaN,0.0,NaN,6.0,1.0,7.0,NaN,10.0,NaN,NaN,1.0,12.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2.0,NaN,4.0,NaN,NaN,NaN,7.0,NaN,NaN,NaN,6.0,NaN,NaN,2.0,2.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,4.0,16.0,NaN,0.0,6.0,NaN,NaN,NaN,0.0,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,2.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0,NaN,NaN,NaN,2.0,2.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.0,0.0,NaN,NaN,NaN,2.0,2.0,NaN,0.0,NaN,NaN,NaN,-1.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,2.0,2.0,NaN,5.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.0,0.0,6.0,6.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,5.0,NaN,NaN,NaN,5.0,NaN,0.0,-1.0,2.0,NaN,2.0,NaN,NaN,1.0,2.0,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,15.0,NaN,NaN,13.0,6.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,8.0,6.0,NaN,2.0,0.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5.0,NaN,NaN,NaN,1.0,2.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,2.0,2.0,NaN,0.0,NaN,NaN,NaN,1.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,0.0,1.0,NaN,2.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,5.0,8.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,6.0,NaN,NaN,NaN,4.0,NaN,9.0,1.0,0.0,NaN,1.0,NaN,NaN,2.0,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,0.0,NaN,5.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,3.0,NaN,NaN,3.0,2.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,7.0,2.0,NaN,2.0,0.0,NaN,NaN,NaN,0.0,10.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.0,NaN,NaN,5.0,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,9.0,NaN,NaN,NaN,2.0,2.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,5.0,6.0,NaN,NaN,NaN,9.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,0.0,0.0,NaN,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,2.0,2.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,10.0,NaN,NaN,NaN,1.0,NaN,2.0,2.0,0.0,NaN,2.0,NaN,NaN,2.0,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,3.0,NaN,NaN,6.0,10.0,NaN,7.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,1.0,12.0,NaN,2.0,0.0,NaN,NaN,NaN,0.0,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,6.0,NaN,NaN,8.0,NaN,NaN,3.0,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,10.0,NaN,NaN,NaN,2.0,10.0,NaN,2.0,NaN,N

In [68]:
pivoted.fillna (0)

player_id,6,11,12,14,16,18,22,23,38,40,42,45,54,62,67,69,70,76,77,78,80,83,84,86,91,93,97,100,106,109,111,113,115,118,122,123,126,127,128,132,134,138,141,142,143,148,150,154,155,159,162,164,166,168,172,184,187,191,198,199,200,202,203,209,210,211,212,213,215,217,219,221,224,225,230,231,232,233,234,235,239,240,245,247,251,252,254,255,256,257,259,262,265,266,268,270,271,274,276,277,278,280,282,284,296,298,300,301,302,305,306,313,315,326,336,338,340,342,343,345,351,353,357,359,362,364,366,367,368,370,376,388,390,394,396,403,409,411,417,419,423,425,431,432,435,437,449,450,465,468,475,493,494,558,570,588,681,696,697,699,700,701,702
time_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,0.0,6.0,0.0,0.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,2.0,0.0,10.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,7.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,9.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,8.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,2.0,5.0,0.0,5.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,1.0,1.0,8.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0,7.0,0.0,10.0,0.0,0.0,1.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,6.0,0.0,0.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,16.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,-1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,2.0,2.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,0.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,-1.0,2.0,0.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,15.0,0.0,0.0,13.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,8.0,6.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,1.0,2.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,0.0,1.0,0.0,2.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,6.0,0.0,0.0,0.0,4.0,0.0,9.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,7.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,0.0,0.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,9.0,0.0,0.0,0.0,2.0,2.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,5.0,6.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,0.0,0.0,0.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,3.0,0.0,0.0,6.0,10.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0,12.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,0.0,0.0,8.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,10.0,0.0,0.0,0.0,2.0,10.0,0.0,2.0,0.0,0

In [74]:
pivoted = pivoted[150:]

In [75]:
mu = expected_returns.mean_historical_return (pivoted)
S = risk_models.sample_cov (pivoted)

In [76]:
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

ValueError: P must be symmetric/Hermitian.